In [1]:
import tensorflow as tf
from model import BlazePose
from config import best_pre_train

In [2]:
from data import train_dataset, test_dataset
loss_func = tf.keras.losses.BinaryCrossentropy()
# from data import finetune_train as train_dataset
# from data import finetune_validation as test_dataset
# loss_func = tf.keras.losses.MeanSquaredError()

Reading dataset...
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Generating training and testing data batches...
Done.


In [3]:
model = BlazePose()
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

In [4]:
# checkpoint_path_heatmap = "checkpoints_heatmap/cp-{epoch:04d}.ckpt"
# model.load_weights(checkpoint_path_heatmap.format(epoch=best_pre_train))

In [16]:
def grad(model, inputs, targets):
    with tf.GradientTape() as tape:
        loss_value = loss_func(y_true=targets, y_pred=model(inputs))
    return loss_value, tape.gradient(loss_value, model.trainable_variables, unconnected_gradients=tf.UnconnectedGradients.ZERO)

In [17]:
for layer in model.layers[16:24]:
    layer.trainable = False
# for layer in model.layers[0:16]:
#     layer.trainable = False

In [18]:
epoch_loss_avg = tf.keras.metrics.Mean()
epoch_accuracy = tf.keras.metrics.MeanSquaredError()

In [19]:
# Training loop
for x, y in train_dataset:
    # Optimize
    print(y.shape)
#     print(tf.expand_dims(y[:, :, 2], axis=2).shape)
#     print(tf.concat([tf.expand_dims(y[:, :, 2], axis=2), tf.expand_dims(y[:, :, 2], axis=2)], axis=2).shape)
#     loss_value, grads = grad(model, x, (y[:, :, 0:2], tf.concat([tf.expand_dims(y[:, :, 2], axis=2), tf.expand_dims(y[:, :, 2], axis=2)], axis=2)))
    loss_value, grads = grad(model, x, y)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    # Add current batch loss
    epoch_loss_avg(loss_value)
    # Calculate error from Ground truth
#     epoch_accuracy((y[:, :, 0:2], tf.concat([tf.expand_dims(y[:, :, 2], axis=2), tf.expand_dims(y[:, :, 2], axis=2)], axis=2)), model(x))
    epoch_accuracy(y, model(x))

(64, 128, 128, 14)
(64, 128, 128, 14)
(64, 128, 128, 14)
(64, 128, 128, 14)
(64, 128, 128, 14)
(64, 128, 128, 14)
(64, 128, 128, 14)
(64, 128, 128, 14)
(64, 128, 128, 14)
(64, 128, 128, 14)
(64, 128, 128, 14)
(64, 128, 128, 14)
(64, 128, 128, 14)
(64, 128, 128, 14)
(64, 128, 128, 14)
(40, 128, 128, 14)


In [15]:
model.summary()

Model: "blaze_pose"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              multiple                  672       
_________________________________________________________________
sequential (Sequential)      (None, 128, 128, 24)      840       
_________________________________________________________________
sequential_1 (Sequential)    (None, 128, 128, 24)      840       
_________________________________________________________________
blaze_block (BlazeBlock)     multiple                  11136     
_________________________________________________________________
blaze_block_1 (BlazeBlock)   multiple                  50976     
_________________________________________________________________
blaze_block_2 (BlazeBlock)   multiple                  233088    
_________________________________________________________________
blaze_block_3 (BlazeBlock)   multiple                  6

In [10]:
# a = tf.keras.models.Sequential([
#             tf.keras.layers.GlobalAveragePooling2D(),
#             # shape = (1, 1, 1, 288)
#             tf.keras.layers.Dense(units=2*14, activation=None),
#             # tf.keras.layers.Reshape((num_joints, 2))
#         ])

In [11]:
# import numpy as np
# x = np.arange(1152).reshape(1, 2, 2, 288)

In [12]:
# coordinates = a(x)

In [13]:
# coordinates.shape

In [14]:
for layer in model.layers:
    print(layer)